In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from matplotlib import pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import xml.etree.ElementTree as ET
import re
from unicodedata import normalize
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pandas_summary import DataFrameSummary
%matplotlib
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)
import os
import warnings
warnings.simplefilter(action = 'ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
import seaborn as sns
sns.set(style="ticks", color_codes=True, font_scale=1.5)
color = sns.color_palette()
sns.set_style('darkgrid')
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
import pylab 
from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox
from scipy.special import boxcox1p
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import RobustScaler, PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectKBest, RFECV, SelectFromModel
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.feature_extraction import FeatureHasher
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression, OrthogonalMatchingPursuit, Lasso, LassoLarsIC, ElasticNet, ElasticNetCV
from sklearn.linear_model import SGDRegressor, PassiveAggressiveRegressor, HuberRegressor, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, cross_val_predict, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Abrir tabla con datos internos y externos
tablin = pd.read_csv(r'C:\Users\Marta\Scripts - Presentación\tablin_datos_externos.csv', 
                   sep=';', engine="python", decimal=',', encoding="latin1")

In [3]:
#Procesar las fechas
fechas = list(tablin.Fecha.drop_duplicates())
dictionary_dates_date_to_num = {}
dictionary_dates_num_to_date = {}
for date_iter in range(len(fechas)):
    dictionary_dates_date_to_num[fechas[date_iter]] = date_iter
    dictionary_dates_num_to_date[date_iter] = fechas[date_iter]

dictionary_dates_num_to_date[56] = '01/01/2019'
dictionary_dates_num_to_date[57] = '01/04/2019'
dictionary_dates_num_to_date[58] = '01/07/2019'
dictionary_dates_num_to_date[59] = '01/10/2019'
dictionary_dates_num_to_date[60] = '01/01/2020'
dictionary_dates_num_to_date[61] = '01/04/2020'
dictionary_dates_num_to_date[62] = '01/07/2020'
dictionary_dates_num_to_date[63] = '01/10/2020'

In [4]:
#Eliminar outliers
tablin= tablin[np.abs(tablin['Hipotecas']-tablin['Hipotecas'].mean()) <= (3*tablin['Hipotecas'].std())] 

#Log transformation
tablin['Hipotecas_log'] = np.log(tablin['Hipotecas'])
tablin['Tasa desempleo_log'] = np.log(tablin['Tasa desempleo'])

In [5]:
#Eliminar nulos y duplicados
tablin=tablin.dropna()
tablin= tablin.drop_duplicates()

In [6]:
def parseo_fecha_cuatrimeste(row):
    if row[1] == 'Primero':
        date = '01/01/'
    elif row[1] == 'Segundo':
        date = '01/04/'
    elif row[1] == 'Tercero':
        date = '01/07/'
    elif row[1] == 'Cuarto':
        date = '01/10/'
    return date+str(row[0])

In [7]:
## Procesar
tablin = tablin[tablin['SUPADAPT']>1]
tablin['Index'] = tablin.index 
data_pred= tablin
data_pred = data_pred.drop_duplicates()

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_pred['Fecha_num']=le.fit_transform(data_pred['Fecha'])

In [9]:
#Recodificar
data_pred= data_pred.replace('SI', '1')
data_pred= data_pred.replace('NO', '0')
data_pred= data_pred.replace('LIBRE', 'Libre' )
data_pred= data_pred.replace('ESPECIAL', 'Especial')

In [10]:
data_pred.shape

(123537, 36)

In [11]:
## Eliminar duplicados y nulos

data_pred = data_pred.drop_duplicates()
data_pred= data_pred.dropna() 

In [12]:
#Procesar EDIF_FECHA_CONSTRUCCION 

data_pred.loc[:,'EDIF_FECHA_CONSTRUCCION'] = data_pred['EDIF_FECHA_CONSTRUCCION'].apply(lambda s: (s[5:9]))
data_pred['EDIF_FECHA_CONSTRUCCION']= data_pred.loc[:,['EDIF_FECHA_CONSTRUCCION']].replace("", np.nan)
data_pred= data_pred.dropna() 

#Convertimos la fecha de construccion a una variable de antigüedad de la vivienda
data_pred['EDIF_FECHA_CONSTRUCCION']= data_pred['EDIF_FECHA_CONSTRUCCION'].astype(int)
indexNames = data_pred[(data_pred['EDIF_FECHA_CONSTRUCCION'] < 1800)].index
data_pred.drop(indexNames , inplace=True)
data_pred['EDIF_FECHA_CONSTRUCCION']= data_pred['EDIF_FECHA_CONSTRUCCION'] - 1800

In [13]:
## Crear variable de variación del precio de la vivienda por año
#Ordenar por fecha (quarterly)
data_pred= data_pred.sort_values('Fecha_num')

#Agrupar por fecha y calcular la media del precio de las viviendas
data_pred['Variacion'] = data_pred.groupby('Fecha_num')['IMPTOTAL'].transform('mean')

#Calcular la variacion de la media del precio de las viviendas respecto al trimestre anterior
c= data_pred.loc[:,['Fecha_num', 'Variacion']]
c= c.drop_duplicates()
c['change'] = c['Variacion'].pct_change()
c=c.fillna(0)
data_pred= data_pred.merge(c.drop(['Variacion'],1), how='left', on=['Fecha_num'])
data_pred= data_pred.drop(columns='Variacion')

In [14]:
columns_to_del = ['DESPOBLACI','CODPOSTAL', 'CODPROVINC', 'Hipotecas', 'Tasa desempleo', 'Index']
for col in columns_to_del:
    del data_pred[col]

In [15]:
#Asignar el tipo de variables (las siguientes son strings)
for col in ['EDIF_CALIDAD_CONSTRU_COD','EDIF_EST_CONSERVACION_COD','VIV_ORIENTACION_COD','EDIF_FECHA_CONSTRUCCION']:
    data_pred.loc[:,col] = data_pred.loc[:,col].apply(lambda s: str(s))

In [16]:
## Quitar outliers y guardar la tabla final

data_pred= data_pred[np.abs(data_pred['IMPTOTAL']-data_pred['IMPTOTAL'].mean()) <= (3*data_pred['IMPTOTAL'].std())] 

In [17]:
# Cambiar nombre de algunas columnas
data_pred=data_pred.rename(columns = {'SUPADAPT':'SUPERF_ADOPTADA', 'DESMUNIC': 'Municipio'})

In [18]:
data_pred_index = data_pred.loc[:,['MATRICULA','Fecha_num']].groupby('MATRICULA').idxmax()   #agrupamos por matrícula (la fecha más reciente)
data_pred = data_pred.loc[data_pred_index['Fecha_num']]   #filtramos y nos quedamos sin duplicados de la misma vivienda con distinta fecha 

In [19]:
data_pred.to_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos - copia - Marta\tabla.csv', 
                   sep=';', decimal=',', encoding="latin1", index=False)

In [20]:
data_pred.shape

(84735, 31)